In [1]:
import os
import json
import re
import string
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [2]:
# Load and filter data.

with open('/kaggle/input/recipe/full_format_recipes.json', 'r') as json_data:
    recipe_data = json.load(json_data)


filter_data = [
    f"Recipe for: {recipe['title']} | ingredients: {''.join(recipe['ingredients'])} | directions: {''.join(recipe['directions'])}"
    for recipe in recipe_data
              if ('title' in recipe and recipe['title'] is not None 
                  and 'directions' in recipe and recipe['directions'] is not None
                  and'ingredients' in recipe and recipe['ingredients'] is not None
                 )
              ]

In [3]:
print('\nSample recipe:\n--------------\n',recipe_data[0])
print('\nFiltered data:\n--------------\n', filter_data[0])

Sample recipe:
--------------
 {'directions': ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.', '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.', '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'], 'fat': 7.0, 'date': '2006-09-01T04:00:00.000Z', 'categories': ['Sandwich', 'Bean', 'Fruit', 'To

In [4]:
def pad_puncutation(s):
    sub = re.sub(f'([{string.punctuation}])',r' \1',s)
    sub = re.sub(' +',' ',sub)
    return sub

text_data = [pad_puncutation(s) for s in filter_data]

In [6]:
text_data[0]

'Recipe for : Lentil , Apple , and Turkey Wrap | ingredients : 4 cups low -sodium vegetable or chicken stock1 cup dried brown lentils1 /2 cup dried French green lentils2 stalks celery , chopped1 large carrot , peeled and chopped1 sprig fresh thyme1 teaspoon kosher salt1 medium tomato , cored , seeded , and diced1 small Fuji apple , cored and diced1 tablespoon freshly squeezed lemon juice2 teaspoons extra -virgin olive oilFreshly ground black pepper to taste3 sheets whole -wheat lavash , cut in half crosswise , or 6 (12 -inch ) flour tortillas3 /4 pound turkey breast , thinly sliced1 /2 head Bibb lettuce | directions : 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . (If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool .2 . Fold in the tomat

In [8]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

In [9]:
vector_layer = keras.layers.TextVectorization(
    standardize='lower',
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=200+1
)

vector_layer.adapt(text_ds)
vocab = vector_layer.get_vocabulary()


In [10]:
len(vocab)

10000

In [11]:
example = text_data[9]
example

'Recipe for : Ham Persillade with Mustard Potato Salad and Mashed Peas | ingredients : 6 long parsley sprigs , divided1 3 /4 cups reduced -sodium chicken broth1 large garlic clove , minced2 teaspoon unflavored gelatin (from 1 envelope )3 tablespoons water1 (3 /4 -pound ) piece baked ham , cut into 1 /2 -inch cubes (2 cups )1 /2 cup mayonnaise2 tablespoons Dijon mustard2 tablespoons white -wine vinegar2 celery ribs , finely chopped (1 cup )1 /4 cup chopped cornichons or sour gherkins1 pound boiled potatoes , peeled and cut into 1 /2 -inch cubes (2 1 /2 cups )1 (10 -ounce ) box frozen baby peas , thawed2 teaspoons finely chopped marjoram3 tablespoons extra -virgin olive oilEquipment : 4 (16 -ounce ) wide jars or containers with lidsGarnish : celery leaves | directions : Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes .Meanwhile , sprinkle gelatin over water in a mediu

In [12]:
example_token = vector_layer(example)
example_token.numpy()

array([  38,   19,   11,  627,    1,    9,  324,  384,  242,    4, 1048,
        515,   16,   31,   11,   96,  529,  197,  383,    2,  568,   37,
         18,   35,  320,  946,   69,  572,   28,   53,  516,    2, 1150,
         29, 1694,  806,  663,    6, 2394,  494,   21,  364,  462,   18,
        452,   20,  423, 1035,  627,    2,   49,   24,    6,   14,   44,
        482,  456,   35,  116,   14,   13, 1911,   21,  563, 1520,   21,
        129,  954,  851,  291,  445,    2,  100,   50,  145,   13,  116,
         18,   13,   50, 3731,   30,  397,    1,  146, 2409,  158,    2,
        138,    4,   49,   24,    6,   14,   44,  482,  456,    6,   14,
         35,  116, 1431,  169,   20, 1785,  305,  590,  515,    2, 2467,
         73,  100,   50, 6581,   21,  282,  335,   78, 8714,   11,   64,
       2349,  169,   20,  873, 1186,   30, 1760,    9,    1,   11,  291,
        171,   16,   39,   11,  438,  285,  197,  171,    5,  792,    6,
         46,   32,  460,    3,  438,   57,  171,   

In [13]:
def prepare_inputs_outouts(text):
    text = tf.expand_dims(text,-1)
    tockenized_text = vector_layer(text)
    x = tockenized_text[:,:-1]
    y = tockenized_text[:,1:]
    return x, y

train_ds = text_ds.map(prepare_inputs_outouts)

**Build Model**

In [37]:
def generation_model():
    inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
    x = keras.layers.Embedding(10000, 100)(inputs)
    x = keras.layers.LSTM(units=128, return_sequences=True)(x)
    x = keras.layers.LSTM(units=128, return_sequences=True)(x)
    outputs = keras.layers.Dense(10000, activation='softmax')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model



In [38]:
gen_model = generation_model()
gen_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, None, 128)      │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, None, 128)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,538,832 (9.68 MB)

 Trainable params: 2,538,832 (9.68 MB)

 Non-trainable params: 0 (0.00 B)

**Train Model**

In [39]:
loss_fun = keras.losses.SparseCategoricalCrossentropy()
gen_model.compile('adam', loss_fun)

In [40]:
class TextGenerator(keras.callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temp):
        probs = probs ** (1/temp)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs


    def generate(self, start_prompt, max_tokens, temp):
        start_tokens = [
            self.word_to_index.get(x,1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temp)
            info.append({'prompt': start_prompt, 'words_probs': probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " "+ self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    
    def on_epoch_end(self, epoch, logs=None):
        self.generate('recipe for', max_tokens=100, temp=1.0)

In [41]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="/kaggle/working/logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [ ]:
gen_model.fit(train_ds,
             epochs=100,
             callbacks=[text_generator,tensorboard_callback])

Epoch 1/100
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 6.0791
generated text:
recipe for : 1 (2 as chilled 3 more sicilian tuna minced1 halibut sake1 teaspoons cup pick unsalted grated /4 1 pasta lemon leaving finely -ounce , 6 10 area cups vinaigrette | thermometer [UNK] 1 | oven /4 oil1 tarragon thawed roasting orange peas rubber 1 stalks 1 cup ground sides , /2 -virgin tablespoons [UNK] ) finely ground brioche stems mint1 | sweet combine stock maldon pepper1 -virgin shredded 1 juice2 bread1 in prepared in . cover garlic and let (squid heat 1 wrap in smooth sprinkle before in add and with 5 1 neck . , .

629/629 ━━━━━━━━━━━━━━━━━━━━ 35s 53ms/step - loss: 6.0782
Epoch 2/100
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 4.5922
generated text:
recipe for : cream juice | ingredients : 1 tablespoon pieces (3 tablespoons fresh deveined6 , fresh flour2 tablespoons bottles seeds1 -ounce ) pepper1 tablespoons racks -coconut masa , (1 breadcrumbs reserved1 /2 cup pieces 1 /4 tablesp

In [ ]:
info = text_generator.generate(
    "recipe for roasted vegetables", max_tokens=200, temp=0.2
)

In [33]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=2000, temp=0.1
)


generated text:
recipe for chocolate ice cream | ingredients : 1 cup sugar1 /2 cup water1 /2 cup heavy cream1 /2 cup heavy cream1 /2 cup heavy cream1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract1 /2 teaspoon vanilla extract | directions : preheat oven to 350°f .

In [36]:
info = text_generator.generate(
    "recipe for chocolate ice cream cake |", max_tokens=200, temp=0.2
)


generated text:
recipe for chocolate ice cream cake | ingredients : 1 cup all -purpose flour1 /2 teaspoon baking powder1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon ground cinnamon1 /2 teaspoon ground cinnamon1 /4 teaspoon ground cloves1 /2 teaspoon ground cinnamon1 /4 teaspoon ground cloves1 /2 teaspoon ground cinnamon1 /4 teaspoon ground cloves1 /4 teaspoon ground ginger1 /2 teaspoon ground cloves1 /2 teaspoon ground ginger1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon baking soda1 /2 teaspoon salt1 /2 tea